In [ ]:
# 주피터
import os

os.chdir('G:/내 드라이브/projects/NLP-StockMarket/model_fin/')

In [ ]:
# 코랩
from google.colab import drive
drive.mount("/content/drive")

import os
path = '/content/drive/My Drive/projects/NLP-StockMarket/model_fin/' # 본인 구글 드라이브 계정마다 살짝씩 다를 수도 있음
os.chdir(path)

In [ ]:
!pip install konlpy   # 코랩에서만 실행

# 라이브러리

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from datetime import timedelta
from konlpy.tag import *

from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score, confusion_matrix, classification_report, log_loss
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

# 종목명 선택 및 뉴스,토론방, 유튜브 데이터 통합

In [3]:
# LG화학, 삼성SDI, SK이노베이션, 고려아연, 포스코케미칼
stock_name = '삼성SDI'

In [4]:
naver_news = pd.read_csv('./data/refined_naver_news.csv', index_col = 0)
daum_news = pd.read_csv('./data/refined_daum_news.csv', index_col = 0)
naver_talks = pd.read_csv(f'./data/refined_naver_talks_{stock_name}.csv', index_col = 0)
daum_talks = pd.read_csv(f'./data/refined_daum_talks_{stock_name}.csv', index_col = 0)
youtube = pd.read_csv(f'./data/refined_youtube_{stock_name}.csv', index_col = 0)

In [5]:
# 데이터 통합
news_df = pd.concat([naver_news, daum_news, naver_talks, daum_talks ,youtube])

# 'Date' 타입이 int 이므로 datetime으로 변환
news_df['Date'] = pd.to_datetime(news_df['Date'].astype(str))

# 합쳐진 데이터들의 인덱스 재설정
news_df.sort_values('Date', ignore_index=True, inplace=True)

In [6]:
# 2021년 주식 데이터가 1월 4일부터 있어서 슬라이싱
news_df = news_df[news_df[news_df['Date']== '2021-01-04'].index[0] : ]
news_df.head(2)

,Date,Title
639,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결
640,2021-01-04,환율 하락 전환 1086 2 감소0 1원


# 주가 데이터

In [7]:
stock_df = pd.read_csv(f'./data/{stock_name}_주가_데이터.csv', usecols = ['일자','등락률'])
stock_df['일자'] = pd.to_datetime(stock_df['일자'])
stock_df.head(2)

,일자,등락률
0,2021-01-04,6.85
1,2021-01-05,2.24


In [8]:
start = str(stock_df.iloc[0, 0])
end = str(stock_df.iloc[-1, 0])
print(start)
print(end)

2021-01-04 00:00:00
2022-06-30 00:00:00


# 데이터 프레임 합치기

In [9]:
## 뉴스일자 조정(예측대상(주가)의 일자와 맞추기 위해)
news_df['일자'] = news_df['Date'] + timedelta(days=1)

In [10]:
df = news_df.merge(stock_df)
df.columns = [df.columns[0], df.columns[1], '주가의 날짜', '등락률']
df.drop_duplicates('Title', inplace = True, ignore_index = True)  # 기사제목 중복 제거
df['Title'] = df['Title'].astype(str)
print(len(df))
df.head()

429241


,Date,Title,주가의 날짜,등락률
0,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24
1,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24
2,2021-01-04,코스피 1 03p 0 04 오른 2874 50 출발 원 달러 환율 1 2원 ...,2021-01-05,2.24
3,2021-01-04,韓증시 사상최고치 시작 개장식선 안정적 시장운영에 방점,2021-01-05,2.24
4,2021-01-04,SK바이오팜 아벨 지분 매각으로 5천500만 달러 자본이득,2021-01-05,2.24


# New 감성사전 load
- SP+word 합친 감성사전

In [11]:
sentiment_csv = pd.read_csv('./sentiment dictionary new.csv', index_col = 0, usecols=[0,1,2])
sentiment_csv.head()

,Pos,Neg
0,경신,매도
1,상승세,조정
2,오른,감소
3,강화,전쟁
4,상승,개발


In [12]:
pos_li = sentiment_csv['Pos'].dropna().values
neg_li = sentiment_csv['Neg'].dropna().values

# 감성 지수 계산하는 binary_sentimental_score()

In [13]:
def binary_sentimental_score(df):
    # 입력받은 데이터프레임 복사 및 컬럼 추가
    df_result = df.copy()
    
    df_result['Pos'] = 0
    df_result['Neg'] = 0
    
    # 감성 지수는 긍정 : 1, 부정 : -1, 해당 데이터 제외 : 999
    df_result['감성지수'] = 999 
    
    # 등락률에 의해 결정되는 updown    
    df_result['updown'] = 0
    df_result.loc[df_result.query('등락률 >= 0').index, 'updown'] = 1
    df_result.loc[df_result.query('등락률 < 0').index, 'updown'] = -1
    
################################################################################################
    # 0  : 없음, 1: 있음
    # 감성 사전에 따른 텍스트 검출
    print('긍정 단어 검색중')
    for pos in tqdm(pos_li) :
        str_expr = f"Title.str.contains('{pos}')"
        df_result.loc[df_result.query(str_expr).index, 'Pos'] = 1
    
    print('부정 단어 검색중')
    for neg in tqdm(neg_li) :
        str_expr = f"Title.str.contains('{neg}')"
        df_result.loc[df_result.query(str_expr).index, 'Neg'] = 1
    
################################################################################################
    # 긍정 단어만이 검출되면 긍정
    df_result.loc[df_result.query('Pos == 1 and Neg == 0').index, '감성지수'] = 1
    
    # 부정 단어만이 검출되면 부정
    df_result.loc[df_result.query('Pos == 0 and Neg == 1').index, '감성지수'] = -1
    
    # 긍정, 부정 단어가 둘 다 있으면 전 날 또는 당일 주가의 등락률을 보고 결정
    print('긍정 부정 둘 다 있는 경우 처리중')
    for i in tqdm(df_result.loc[df_result.query('Pos == 1 and Neg == 1').index].index) : 
        # 등락률에 따라 감성지수를  정하는 check
        check = 999
        
        # 해당 Title의 어제 주가가 있으면 선택
        if sum(df_result.loc[i,'Date'] - timedelta(days = 1) == stock_df['일자']) == 1 :  
            check = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = 1)]['등락률'].values[0]
        
        # 어제 주가는 없지만 당일이 있으면 당일을 선택
        elif sum(df_result.loc[i,'Date'] == stock_df['일자']) == 1 :  
            check = stock_df[stock_df['일자'] == df_result.loc[i,'Date']]['등락률'].values[0]

        # 어제와 오늘의 주가도 없다면 이전의 주가를 찾아 탐색
        else :
            j = 2 
            while True :
                if sum(df_result.loc[i,'Date'] - timedelta(days = j) == stock_df['일자']) == 1 :
                    check = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = j)]['등락률'].values[0]
                    break
                j += 1
        
        if check >= 0 :
            df_result.loc[i,'감성지수'] = 1
        else :
            df_result.loc[i,'감성지수'] = -1
    
################################################################################################    
    # 감성지수가 긍정 부정 둘 다 아니면 삭제
    df_result = pd.concat([df_result.loc[df_result.query('감성지수 != 999').index]], ignore_index = True)
    
    return df_result

In [14]:
df_result = binary_sentimental_score(df)

긍정 단어 검색중


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:00<00:00,  5.55it/s]


부정 단어 검색중


100%|██████████████████████████████████████████████████████████████████████████████| 1323/1323 [03:56<00:00,  5.60it/s]


긍정 부정 둘 다 있는 경우 처리중


100%|██████████████████████████████████████████████████████████████████████████| 88693/88693 [00:52<00:00, 1685.20it/s]


In [15]:
df_result

,Date,Title,주가의 날짜,등락률,Pos,Neg,감성지수,updown
0,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24,1,0,1,1
1,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24,0,1,-1,1
2,2021-01-04,코스피 1 03p 0 04 오른 2874 50 출발 원 달러 환율 1 2원 ...,2021-01-05,2.24,1,0,1,1
3,2021-01-04,韓증시 사상최고치 시작 개장식선 안정적 시장운영에 방점,2021-01-05,2.24,1,1,1,1
4,2021-01-04,SK바이오팜 아벨 지분 매각으로 5천500만 달러 자본이득,2021-01-05,2.24,1,1,1,1
...,...,...,...,...,...,...,...,...
353393,2022-06-29,새빗켐 증권신고서 제출 코스닥 상장 본격화,2022-06-30,-6.67,1,0,1,-1
353394,2022-06-29,SK바사 토종 백신 1호 식약처 승인 소식에 3 대 강세,2022-06-30,-6.67,1,0,1,-1
353395,2022-06-29,장중시황 코스피 1 4 내린 2389선 경기 침체 우려에 약세,2022-06-30,-6.67,0,1,-1,-1
353396,2022-06-29,코스피 외인 기관 매도세 확대 코스닥 낙폭 축소,2022-06-30,-6.67,1,1,1,-1


In [16]:
df_result['updown'].value_counts()

 1    199641
-1    153757
Name: updown, dtype: int64

In [17]:
df_result['감성지수'].value_counts()

 1    240051
-1    113347
Name: 감성지수, dtype: int64

# 예측 모델 적용

In [18]:
# 형태소 분석을 위한 함수
def tokenizer(text):
    okt = Okt()
    return okt.morphs(text)

In [19]:
# X : title, y : price
def data_split(X, y):
    # 수집한 데이터 읽어오기
    
    # 학습셋, 테스트셋 분리
    X_list = X.tolist()
    y_list = y.tolist()
    
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, shuffle = True, test_size = 0.2)
    
    return X_train, X_test, y_train, y_test

In [20]:
X_train, X_test, y_train, y_test = data_split(df_result['Title'], df_result['감성지수'])

In [21]:
tokenize_X_train = TfidfVectorizer(tokenizer = tokenizer).fit_transform(X_train)
tokenize_X_test = TfidfVectorizer(tokenizer = tokenizer).fit_transform(X_test)

# 로지스틱 회귀 그리드 서치

In [22]:
def parameter_serach(penalty, solvers) :
    best_score = 0
    Cs = [0.001, 0.01, 0.1, 1, 10, 100]
        
    kfold = KFold(3, shuffle = True)

    for s in solvers :
        scores_mean = []
        for C in tqdm(Cs) :
            lm = LogisticRegression(penalty = penalty, C = C, solver = s, multi_class = 'ovr')
            scores = cross_val_score(lm, tokenize_X_train, y_train, cv = kfold)

            score = np.mean(scores)
            scores_mean.append(score)

            if score > best_score:
                best_score = score
                best_parameters = {'penalty' : penalty, 'C': C, 'solver': s, 'best_score' : best_score}

    return best_parameters 

In [23]:
# saga도 쓸 수 있지만 C값이 1 이상부터 시간이 너무 오래 걸림(분 단위)
l1_best_parameters = parameter_serach('l1', ['liblinear'])

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:55<00:00, 19.20s/it]


In [24]:
l2_best_parameters = parameter_serach('l2', ['liblinear', 'sag', 'saga'])

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [02:10<00:00, 21.76s/it]


#### 로지스틱 회귀의 최적 파라미터 확인¶

In [25]:
print(f"penalty\t\t:\t{l1_best_parameters['penalty']}")
print(f"C\t\t:\t{l1_best_parameters['C']}")
print(f"solver\t\t:\t{l1_best_parameters['solver']}")
print(f"best_score\t:\t{l1_best_parameters['best_score']}")

penalty		:	l1
C		:	10
solver		:	liblinear
best_score	:	0.8763573637184431


In [26]:
print(f"penalty\t\t:\t{l2_best_parameters['penalty']}")
print(f"C\t\t:\t{l2_best_parameters['C']}")
print(f"solver\t\t:\t{l2_best_parameters['solver']}")
print(f"best_score\t:\t{l2_best_parameters['best_score']}")

penalty		:	l2
C		:	10
solver		:	saga
best_score	:	0.8788474737013016


In [27]:
if l1_best_parameters['best_score'] > l2_best_parameters['best_score'] :
    best_param = l1_best_parameters
else :
    best_param = l2_best_parameters

In [29]:
best_param

{'penalty': 'l2', 'C': 10, 'solver': 'saga', 'best_score': 0.8788474737013016}

## 파이프라인

In [30]:
# lowercase : 소문자로 반환 여부 lowercase = False, 
# tokenizer : 토크나이저 지정
tfidf = TfidfVectorizer(tokenizer = tokenizer)

In [31]:
logistic = LogisticRegression(penalty = best_param['penalty'],
                              C = best_param['C'],
                              solver = best_param['solver'],
                             multi_class = 'ovr')

In [32]:
pipeline = Pipeline([('tfidf',tfidf), ('classfier', logistic)], verbose = True) 

In [33]:
pipeline.fit(X_train, y_train)

[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 6.2min
[Pipeline] ......... (step 2 of 2) Processing classfier, total=  11.5s


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenizer at 0x00000268E02E3EE0>)),
                ('classfier',
                 LogisticRegression(C=10, multi_class='ovr', solver='saga'))],
         verbose=True)

#### 교차검증

In [34]:
score = cross_val_score(pipeline, X_train, y_train, cv = KFold(3, shuffle = True), scoring='f1_micro')

[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 3.4min
[Pipeline] ......... (step 2 of 2) Processing classfier, total=   4.9s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 3.3min
[Pipeline] ......... (step 2 of 2) Processing classfier, total=   5.2s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total= 3.3min
[Pipeline] ......... (step 2 of 2) Processing classfier, total=   5.0s


In [35]:
print(score)
print(f"모델의 정확도\t:\t{score.mean()}")

[0.87739813 0.8785853  0.87972071]
모델의 정확도	:	0.8785680472057483


In [36]:
y_pred = pipeline.predict(X_test)

In [39]:
y_proba = pipeline.predict_proba(X_test)

## 모델 평가

In [42]:
print(f'accuracy \t:\t {accuracy_score(y_test, y_pred)}')

#'micro', 'macro', 'weighted'
print(f'f1_score \t:\t {f1_score(y_test, y_pred, average = "micro")}')
print(f'f1_score \t:\t {f1_score(y_test, y_pred, average = "macro")}')
print(f'f1_score \t:\t {f1_score(y_test, y_pred, average = "weighted")}')

print(f'R2 \t\t:\t {r2_score(y_test, y_pred)}')

print(f'mse \t\t:\t {mean_squared_error(y_test, y_pred)}')

print(f'rmse \t\t:\t {mean_squared_error(y_test, y_pred, squared=False)}')

print(f'log_loss\t:\t {log_loss(y_test, y_proba)}')

accuracy 	:	 0.8843378607809848
f1_score 	:	 0.8843378607809848
f1_score 	:	 0.8657284982717905
f1_score 	:	 0.8836695277299761
R2 		:	 0.4689410458965433
mse 		:	 0.4626485568760611
rmse 		:	 0.6801827378551011
log_loss	:	 0.2813061368176861


In [43]:
print(confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[18096  3615]
 [ 4560 44409]]
              precision    recall  f1-score   support

          -1       0.80      0.83      0.82     21711
           1       0.92      0.91      0.92     48969

    accuracy                           0.88     70680
   macro avg       0.86      0.87      0.87     70680
weighted avg       0.89      0.88      0.89     70680



## 모델 저장 및 사용

In [44]:
def save_model(model):
    with open('[Model9]pipe_binary.dat', 'wb') as fp:     # 쓰기, 바탕화면에 저장됨
        pickle.dump(model, fp)
    print('저장완료')     # 학습된 모델 저장 완료

In [45]:
save_model(pipeline)

저장완료


In [46]:
with open('[Model9]pipe_binary.dat','rb') as fp:     # 읽기
    pipe = pickle.load(fp)

# Inference

In [47]:
# 모델 사용 함수
def model_prediction():  
    # 객체를 복원, 저장된 모델 불러오기
    with open('[Model9]pipe_binary.dat','rb') as fp:     # 읽기
        pipe = pickle.load(fp)
    while True :
        text = input('뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : \n')
        example = [text]
        # 예측 정확도
        r1 = np.max(pipe.predict_proba(example) * 100)     # 확률값을 구해서 *100
        # 예측 결과 
        r2 = pipe.predict(example)[0]     # 긍정('1'), 부정('-1)

        if text == 'q':
            print("예측을 종료합니다.")
            break
        if r2 == 1 :
            print(f'\n{stock_name} 주가가 상승할 것으로 예상됩니다.')
        elif r2 == -1 :
            print(f'\n{stock_name} 주가가 하락할 것으로 예상됩니다.')
        else : 
            # 이진 분류 모델이라 사실상 실행 되지 않을 라인
            print(f'\n모르겠어요.....')
        print('확률 : %.3f' % r1)
        print('------------------------------------------------\n')

In [48]:
model_prediction()

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
[줌인 이종목] 현대중공업, 실적 증가 기대에 껑충

삼성SDI 주가가 상승할 것으로 예상됩니다.
확률 : 99.737
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
닷새만에 반등했지만…불안한 개미들, 7100억 차익실현

삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 76.720
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
주식거래 뚝…"증권사, 2분기 실적 쇼크"

삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 81.741
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
삼성SDI 흑자 전환

삼성SDI 주가가 상승할 것으로 예상됩니다.
확률 : 99.534
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
삼성SDI 적자 전환

삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 92.785
------------------------------------------------

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 
q
예측을 종료합니다.


- 넣어볼 타이틀 예시  
[줌인 이종목] 현대중공업, 실적 증가 기대에 껑충  
닷새만에 반등했지만…불안한 개미들, 7100억 차익실현  
주식거래 뚝…"증권사, 2분기 실적 쇼크"  
삼성SDI 주식 폭망  
삼성SDI 적자 전환  
삼성SDI 흑자 전환  